<a href="https://colab.research.google.com/github/hbgit/walletstockanalysis/blob/main/ReviewMyWalletStonks_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import json
# Import the plotting library
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px

In [3]:
#PRIO3 = yf.Ticker("PRIO3.SA")
PRIO3 = yf.Ticker("VOO")
print(PRIO3.info)

{'exchange': 'PCX', 'shortName': 'Vanguard S&P 500 ETF', 'longName': 'Vanguard 500 Index Fund', 'exchangeTimezoneName': 'America/New_York', 'exchangeTimezoneShortName': 'EDT', 'isEsgPopulated': False, 'gmtOffSetMilliseconds': '-14400000', 'quoteType': 'ETF', 'symbol': 'VOO', 'messageBoardId': 'finmb_28117396', 'market': 'us_market', 'annualHoldingsTurnover': None, 'enterpriseToRevenue': None, 'beta3Year': 1, 'profitMargins': None, 'enterpriseToEbitda': None, '52WeekChange': None, 'morningStarRiskRating': None, 'forwardEps': None, 'revenueQuarterlyGrowth': None, 'fundInceptionDate': 1283817600, 'annualReportExpenseRatio': None, 'totalAssets': 686500216832, 'bookValue': None, 'fundFamily': 'Vanguard', 'lastFiscalYearEnd': None, 'netIncomeToCommon': None, 'trailingEps': None, 'lastDividendValue': None, 'SandP52WeekChange': None, 'priceToBook': None, 'nextFiscalYearEnd': None, 'yield': 0.0177, 'mostRecentQuarter': None, 'enterpriseValue': None, 'priceHint': 2, 'threeYearAverageReturn': 0.0

**Stock Wallet**

In [4]:
ticket_company_shares = ['ABBV','PEP','PLD','KO','STOR','PRU','MSFT','VZ','T','GOOGL','AMZN']

In [9]:
#earningsGrowth; yield; totalRevenue
metric_company_shares = [
    'marketCap',
    'revenueGrowth',
    'pegRatio',
    'returnOnAssets',
    'returnOnEquity',
    'profitMargins',
    'currentRatio',
    'debtToEquity',
    'payoutRatio',
    'recommendationMean',
    'recommendationKey',
    'totalDebt',
    'sector',
    'dividendYield',
    'averageVolume',
    'totalRevenue'
]

In [10]:
df = pd.DataFrame(index=ticket_company_shares,columns=metric_company_shares)

In [16]:
def get_fundamental_data(df):
  for symbol in df.index:
    get_sym_info = yf.Ticker(symbol)
    print("Getting symbol: " + symbol)    
    for metric in df.columns:        
      #print(get_sym_info.info[metric])
      df.loc[symbol,metric] = get_sym_info.info[metric]    
  return df
      

In [ ]:
# Defining rules to check fundamentals
# gt -> great than >= 
rulesFundmentals_gt_company_shares = {
    'marketCap' : 200000000000,
    'revenueGrowth' : 0.052,
    'pegRatio' : 2.0,
    'returnOnAssets' : 0.08000,
    'returnOnEquity' : 0.20000,
    'profitMargins' : 0.11282,
    'currentRatio' : 1.0,
    'debtToEquity' : 11.00,   
    'averageVolume' : 1700000
}

In [ ]:
df = get_fundamental_data(df)

In [32]:
# Print Bar Chart
def print_bar_chart_from_json(json_file):
  print(json_file)
  #s = pd.Series(json_file, name='DateValue')
  df = pd.DataFrame(json_file.items(), columns=['Ticket', 'Value'])
  fig = px.bar(df, x='Ticket', y='Value',
             color='Value',
             labels={'pop':'population of Canada'}, height=400)
  fig.show()


In [40]:
def check_peformace_last_months(list_symbol):
    # Collect data from the last 6 months  
    #print(list_symbol)  
    data = yf.download(list_symbol,period = "6mo")['Adj Close']
    flag_series = False
   
    if type(data) is pd.Series:
      flag_series = True
      single_symbol = list_symbol[0]
      data = pd.DataFrame({'Date':data.index, single_symbol:data.values})
      #set column as index
      data = data.set_index('Date')

    # https://capitalresearch.com.br/blog/calcular-rentabilidade/
    # Return  
    dict_return = {}  
    for symbol in data.columns:
      if not pd.isna(data[symbol].iloc[0]):
        dict_return[symbol] = ((data[symbol].iloc[-1]/data[symbol].iloc[0])*100)-100
      else:
        dict_return[symbol] = ((data[symbol].iloc[-1]/data[symbol].iloc[1])*100)-100

    print(dict_return)
    worst_return = {}
    flag_has_bad_return = False
    for e in dict_return.keys():
      if dict_return[e] <= -20.00:
        flag_has_bad_return = True
        worst_return[e] = dict_return[e]
    
    #print(json.dumps(worst_return, sort_keys=True, indent=4))
    if flag_has_bad_return:
      print_bar_chart_from_json(worst_return)

    # ----------------- Plot all the close prices  ----------------- 
    # Pandas dataframe. pct_change() function calculates the percentage change between the current and a prior element.
    # Return cumulative product over a DataFrame or Series axis.
    # The next step is to calculate the simple daily returns of the stocks. The formula to calculate this rate is to divide each day by the previous day minus 1.   
    df = (data.pct_change()+1).cumprod()
    if not flag_series:
      df = df.reset_index()
      fig = px.line(df, x="Date", y=df.columns)
      fig.show()
    else:      
      df = df.reset_index()
      fig = px.line(df, x="Date", y=single_symbol)
      fig.show()
    
    

In [36]:
def check_fundamentals_data_shares(df, dict_rules_fundamentals):  
  
  rules_broken_by_symbol = []
  metric_broken_by_symbol = {}

  for symbol in df.index:
    count_broken_rules = 0
    tmp_metrics = []         
    
    for metric in df.columns:                                               
      if metric in dict_rules_fundamentals:                        
        if df.loc[symbol,metric] is not None:                        
          if df.loc[symbol,metric] < dict_rules_fundamentals[metric]:
            count_broken_rules += 1 
            tmp_metrics.append(df.loc[symbol,metric])
            tmp_metrics.append(dict_rules_fundamentals[metric])              
            metric_broken_by_symbol[symbol] = tmp_metrics
          if metric == "payoutRatio":
            if df.loc[symbol,metric] > 1.0:
              count_broken_rules += 1
              tmp_metrics.append(df.loc[symbol,metric])
              tmp_metrics.append(dict_rules_fundamentals[metric])              
              metric_broken_by_symbol[symbol] = tmp_metrics
          if metric == "recommendationKey":
            if df.loc[symbol,metric] ==  "sell":
              count_broken_rules += 1
              tmp_metrics.append(df.loc[symbol,metric])
              tmp_metrics.append(dict_rules_fundamentals[metric])              
              metric_broken_by_symbol[symbol] = tmp_metrics
            elif df.loc[symbol,metric] ==  "hold":
              if df.loc[symbol,"recommendationMean"] > 3.5:
                count_broken_rules += 1
                tmp_metrics.append(df.loc[symbol,"recommendationMean"])
                tmp_metrics.append(3.6)   

      if count_broken_rules >= 3: # number of rules that was broken        
        rules_broken_by_symbol.append(symbol)
      else:
        if symbol in metric_broken_by_symbol.keys():
          metric_broken_by_symbol.pop(symbol) 
    
    #try:      
    #except Exception as e:
    #  print (symbol, 'not found')

  # Useful data
  print("Stocks that broken the rules: ")  
  print(json.dumps(metric_broken_by_symbol, sort_keys=True, indent=4))
  check_peformace_last_months(rules_broken_by_symbol)


In [ ]:
check_fundamentals_data_shares(df, rulesFundmentals_gt_company_shares)

Stocks that broken the rules: 
[*********************100%***********************]  7 of 7 completed
{'PLD': -37.533527131711466, 'VZ': -26.339756167791222}


In [ ]:
# ETFs analysis

In [ ]:
def check_fundamentals_data_etfs(df, dict_rules_fundamentals):  
  
  rules_broken_by_symbol = []
  metric_broken_by_symbol = {}

  for symbol in df.index:
    count_broken_rules = 0
    tmp_metrics = []         
    
    for metric in df.columns:                                               
      if metric in dict_rules_fundamentals:                        
        if df.loc[symbol,metric] is not None:                        
          if df.loc[symbol,metric] < dict_rules_fundamentals[metric]:
            count_broken_rules += 1 
            tmp_metrics.append(df.loc[symbol,metric])
            tmp_metrics.append(dict_rules_fundamentals[metric])              
            metric_broken_by_symbol[symbol] = tmp_metrics
          if metric == "beta3Year":
            if df.loc[symbol,metric] > 1.2:
              count_broken_rules += 1
              tmp_metrics.append(df.loc[symbol,metric])
              tmp_metrics.append(dict_rules_fundamentals[metric])
              metric_broken_by_symbol[symbol] = tmp_metrics
      if count_broken_rules >= 3: # number of rules that was broken        
        rules_broken_by_symbol.append(symbol)
      else:
        if symbol in metric_broken_by_symbol.keys():
          metric_broken_by_symbol.pop(symbol) 
    
    #try:      
    #except Exception as e:
    #  print (symbol, 'not found')

  # Useful data
  print("Stocks that broken the rules: ") 
  print(metric_broken_by_symbol)
  #print(json.dumps(metric_broken_by_symbol, sort_keys=True, indent=4))
  check_peformace_last_months(rules_broken_by_symbol)

In [ ]:
# https://www.investopedia.com/terms/b/beta.asp
# Note pegratio and others were removed
# they have none values
metric_ETFs = [
    'quoteType',
    'beta3Year',
    'trailingAnnualDividendYield',
    'yield', 
    'marketCap', 
    'totalAssets',
    'averageVolume',
    'category'
]

In [ ]:
# Defining rules to check fundamentals
# gt -> great than >= 
rulesFundmentals_gt_etfs = {
    'beta3Year' : 1.1,
    'trailingAnnualDividendYield' : 0.011,
    'yield' : 0.011,
    'totalAssets' : 4730926080,
    'averageVolume' : 621942
}

In [ ]:
ticket_etfs_shares = ['VOO', 'VNQ', 'CIBR']

In [ ]:
df_etfs = pd.DataFrame(index=ticket_etfs_shares,columns=metric_ETFs)

In [ ]:
df_etfs = get_fundamental_data(df_etfs)

Getting symbol: VOO
Getting symbol: VNQ
Getting symbol: CIBR


In [ ]:
df_etfs

,quoteType,beta3Year,trailingAnnualDividendYield,yield,marketCap,totalAssets,averageVolume,category
VOO,ETF,1,0.01581,0.0177,None,686500216832,4420334,Large Blend
VNQ,ETF,1.04,0.042662,0.0389,None,61992325120,5060076,Real Estate
CIBR,ETF,1.04,0,0.0079,None,4730926080,623871,Technology


In [ ]:
check_fundamentals_data_etfs(df_etfs, rulesFundmentals_gt_etfs)

Stocks that broken the rules: 
[*********************100%***********************]  1 of 1 completed
{'CIBR': -20.984107379228092}


In [ ]:
# https://kernc.github.io/backtesting.py/doc/examples/Parameter%20Heatmap.html
# https://towardsdatascience.com/better-heatmaps-and-correlation-matrix-plots-in-python-41445d0f2bec
# 

Analyzing Brasil Stonks

In [13]:
# 'BOVA11.SA' 'IVVB11.SA'
ticket_BRA_shares = ['ABEV3.SA','ASAI3.SA','BPAC11.SA','GOAU4.SA','ITUB4.SA','PETR4.SA','PRIO3.SA','RANI3.SA','TAEE11.SA','TRPL4.SA','VALE3.SA']

In [17]:
df_bra = pd.DataFrame(index=ticket_BRA_shares,columns=metric_company_shares)

In [18]:
df_bra = get_fundamental_data(df_bra) #error sector

Getting symbol: ABEV3.SA
Getting symbol: ASAI3.SA
Getting symbol: BPAC11.SA
Getting symbol: GOAU4.SA
Getting symbol: ITUB4.SA
Getting symbol: PETR4.SA
Getting symbol: PRIO3.SA
Getting symbol: RANI3.SA
Getting symbol: TAEE11.SA
Getting symbol: TRPL4.SA
Getting symbol: VALE3.SA


In [29]:
df_bra.sort_values('debtToEquity')

,marketCap,revenueGrowth,pegRatio,returnOnAssets,returnOnEquity,profitMargins,currentRatio,debtToEquity,payoutRatio,recommendationMean,recommendationKey,totalDebt,sector,dividendYield,averageVolume,totalRevenue
ABEV3.SA,235755061248,0.145,4.67,0.08365,0.1676,0.17603,1.128,4.241,0.7109,2.6,hold,3714621952,Consumer Defensive,0.0456,25403492,76931588096
GOAU4.SA,12161467392,0.244,None,0.18723,0.40275,0.06354,2.489,31.61,0.4845,2,buy,13705967616,Basic Materials,0.2092,8211420,82332590080
VALE3.SA,349016522752,-0.368,0.02,0.16793,0.50592,0.41553,1.322,43.861,0.5717,2.3,buy,84964999168,Basic Materials,0.1045,35912803,249432997888
TRPL4.SA,17406314496,0.103,0.14,0.08117,0.18785,0.47454,7.491,49.838,0.3457,3.2,hold,7991869952,Utilities,0.1225,1908619,5961070080
PETR4.SA,466188828672,0.544,0.12,0.17079,0.41849,0.2842,1.313,67.975,0.7558,2.2,buy,280637014016,Energy,0.3059,89055306,568385011712
PRIO3.SA,28745629696,1.335,None,0.1578,0.40932,0.46822,7.52,71.551,0,1.8,buy,4810321920,Energy,None,14009298,5270664192
TAEE11.SA,41650049024,-0.124,-1.59,0.10295,0.32042,0.6602,2.322,106.664,None,3.8,underperform,7727601152,Utilities,0.0813,1925731,3359247104
RANI3.SA,2250944512,0.064,None,0.13197,0.3754,0.21248,2.787,109.241,0.2769,2.2,buy,1127901056,Consumer Cyclical,0.0765,1381473,1683387008
ASAI3.SA,24048490496,0.211,0.41,0.06591,0.69261,0.03609,1.079,576.307,0.0928,1.9,buy,17197000704,Consumer Defensive,0.0158,11440057,43892998144
BPAC11.SA,92865077248,1.915,None,0.02206,0.1985,0.33815,None,None,None,2.2,buy,183619256320,Financial Services,0.0095,10828595,21021282304


In [30]:
# Defining rules to check fundamentals
# gt -> great than >= 
rulesFundmentals_gt_company_shares_BR = {
    'marketCap' : 2000000000,
    'revenueGrowth' : 0.06,
    'pegRatio' : 0.12,
    'returnOnAssets' : 0.02,
    'returnOnEquity' : 0.18,
    'profitMargins' : 0.2,
    'currentRatio' : 1.3,
    'debtToEquity' : 31.61,   
    'averageVolume' : 1900000
}

In [41]:
check_fundamentals_data_shares(df_bra, rulesFundmentals_gt_company_shares_BR)

Stocks that broken the rules: 
{
    "ABEV3.SA": [
        0.1676,
        0.18,
        0.17603001,
        0.2,
        1.128,
        1.3,
        4.241,
        31.61
    ]
}
[*********************100%***********************]  1 of 1 completed
{'ABEV3.SA': -0.9259282035872616}
